In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import getpass
import os
import shutil
import ipywidgets as widgets
from IPython.display import display

In [2]:

# Prompt user for AGOL credentials
username = input("Enter your AGOL username: ")
password = getpass.getpass("Enter your AGOL password: ")

# Connect to AGOL
gis = GIS("https://www.arcgis.com", username, password)


In [3]:

# Provide the AGOL item ID
item_id = "caaecc422e244897ad5a9d5e0ecf156b"

# Get the item
item = gis.content.get(item_id)

# Create a dropdown widget for selecting the layer
layer_options = [(layer.properties.name, i) for i, layer in enumerate(item.layers)]
layer_dropdown = widgets.Dropdown(
    options=layer_options,
    description='Select Layer:',
    disabled=False,
)

def on_layer_change(change):
    global feature_layer
    feature_layer = item.layers[change['new']]
    print(f"Selected layer: {feature_layer.properties.name}")

layer_dropdown.observe(on_layer_change, names='value')

# Display the dropdown widget
display(layer_dropdown)

# Create a directory to save attachments
output_dir = "attachments"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Dropdown(description='Select Layer:', options=(('admin_0', 0), ('extreme_events', 1)), value=0)

Selected layer: extreme_events


In [6]:
# Function to download attachments
def download_attachments(feature_layer, output_dir):
    # Query all features
    features = feature_layer.query(where="1=1", out_fields="*").features

    for feature in features:
        object_id = feature.attributes['objectid']
        country_label = feature.attributes['country_label_2']
        globalid = feature.attributes['globalid']
        attachments = feature_layer.attachments.get_list(object_id)
        
        for attachment in attachments:
            attachment_id = attachment['id']
            attachment_name = attachment['name']
            final_attachment_path = os.path.join(output_dir, f"{country_label}_{globalid}_{attachment_name}")
            
            # Check if the attachment already exists
            if not os.path.exists(final_attachment_path):
                temp_dir = os.path.join(output_dir, f"temp_{object_id}")
                
                # Create a temporary directory to download the attachment
                if not os.path.exists(temp_dir):
                    os.makedirs(temp_dir)
                
                # Download the attachment to the temporary directory
                feature_layer.attachments.download(oid=object_id, attachment_id=attachment_id, save_path=temp_dir)
                
                # Move the attachment from the temporary directory to the output directory with the new name
                temp_attachment_path = os.path.join(temp_dir, attachment_name)
                shutil.move(temp_attachment_path, final_attachment_path)
                
                # Remove the temporary directory
                shutil.rmtree(temp_dir)
                
                print(f"Downloaded {attachment_name} for feature {object_id} as {final_attachment_path}")
            else:
                print(f"Attachment {attachment_name} for feature {object_id} already exists as {final_attachment_path}")

# Download attachments
download_attachments(feature_layer, output_dir)

Attachment Kaire-Hendrikson-Torgu-Moisakula-Sorve-poolsaar2-768x576.jpg for feature 167 already exists as attachments\Estonia_766830a8-70a9-4505-8e1f-44b1ee6d1981_Kaire-Hendrikson-Torgu-Moisakula-Sorve-poolsaar2-768x576.jpg
Attachment Kaire-Hendrikson-Torgu-Moisakula-Sorve-poolsaar-768x576.jpg for feature 167 already exists as attachments\Estonia_766830a8-70a9-4505-8e1f-44b1ee6d1981_Kaire-Hendrikson-Torgu-Moisakula-Sorve-poolsaar-768x576.jpg
Attachment press_20220914.pdf for feature 211 already exists as attachments\Japan_1a0ef617-0b09-46a5-8c73-f19f37415a4b_press_20220914.pdf
Attachment press_20230928.pdf for feature 211 already exists as attachments\Japan_1a0ef617-0b09-46a5-8c73-f19f37415a4b_press_20230928.pdf
Attachment Summary of extreme and unusual weather 2023 Hong Kong China.docx for feature 272 already exists as attachments\Hong Kong, China_167bdcbc-2506-4b96-8399-e36147f35b3d_Summary of extreme and unusual weather 2023 Hong Kong China.docx
Attachment Italian_Climate_bulletin_2